In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [12]:
import xgboost as xgb
import numpy as np
import pandas as pd

from datetime import datetime
from xgboost import XGBClassifier
from google.cloud import bigquery
from google.cloud.bigquery import dbapi;

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score


In [4]:
def getQuery(proc_code, features):
    """
    Dynamically create feature matrix query using features in param list 'features'
    """
    #values
    lab1_key, lab2_key, lab3_key, lab4_key, lab5_key, \
    lab6_key, lab7_key, lab8_key, lab9_key, lab10_key = features[0]
    
    dx1, dx2, dx3, dx4, dx5, dx6, dx7, dx8, dx9, dx10 = features[1]
    
    hx1, hx2, hx3 = features[2]
    
    med1, med2, med3, med4, med5, med6, med7, med8, med9, med10 = features[3]
    
    for ft in features:
        if None in ft: return None
    
    #names
    dx_name1, dx_name2, dx_name3, dx_name4, dx_name5, dx_name6, dx_name7, \
    dx_name8, dx_name9, dx_name10 = ["dx" + str(n) for n in range(1, 11)]
    
    hx_name1, hx_name2, hx_name3 = ["hx" + str(n) for n in range(1, 4)]
    
    med_name1, med_name2, med_name3, med_name4, med_name5, med_name6, \
    med_name7, med_name8, med_name9, med_name10 = ["med" + str(n) for n in range(1, 11)]
    
    query = """
WITH

--Generate the labels table (1 if result_flag is true and all other columns are null, else 0)
label_labs AS 
(
  SELECT
    anon_id, taken_time, result_time,
    max(case when (lab_result.result_in_range_yn is null and lab_result.result_flag is not null) then 1 else 0 end) as label

  FROM
    shc_core.lab_result
  WHERE
    proc_code = '""" + proc_code + """' and not (reference_low is null and reference_high is null)
  GROUP BY
    anon_id, taken_time, result_time
),

--identify all cases when cohort (patients that have taken the target lab test) had one of the top 10 diagnoses
--before the target lab test was taken
labels_dx AS 
  --note that the naming tells you what tables have been merged
(
  SELECT *
  FROM
  (
    SELECT
      anon_id as dx_anon_id, start_date_utc,
      case when (diagnosis_code.dx_name like '%""" + dx1 + """%') then 1 else 0 end as """ + dx_name1 + """_hist,
      case when (diagnosis_code.dx_name like '%""" + dx2 + """%') then 1 else 0 end as """ + dx_name2 + """_hist,
      case when (diagnosis_code.dx_name like '%""" + dx3 + """%') then 1 else 0 end as """ + dx_name3 + """_hist,
      case when (diagnosis_code.dx_name like '%""" + dx4 + """%') then 1 else 0 end as """ + dx_name4 + """_hist,
      case when (diagnosis_code.dx_name like '%""" + dx5 + """%') then 1 else 0 end as """ + dx_name5 + """_hist,
      case when (diagnosis_code.dx_name like '%""" + dx6 + """%') then 1 else 0 end as """ + dx_name6 + """_hist,
      case when (diagnosis_code.dx_name like '%""" + dx7 + """%') then 1 else 0 end as """ + dx_name7 + """_hist,
      case when (diagnosis_code.dx_name like '%""" + dx8 + """%') then 1 else 0 end as """ + dx_name8 + """_hist,
      case when (diagnosis_code.dx_name like '%""" + dx9 + """%') then 1 else 0 end as """ + dx_name9 + """_hist,
      case when (diagnosis_code.dx_name like '%""" + dx10 + """%') then 1 else 0 end as """ + dx_name10 + """_hist
    FROM
      shc_core.diagnosis_code
  ) AS dx
  RIGHT JOIN 
  (
    SELECT *
    FROM label_labs
  ) AS labels
  ON dx.dx_anon_id = labels.anon_id
  AND CAST(dx.start_date_utc as DATETIME) < CAST(labels.taken_time as DATETIME)
),

--merge diagnoses results (1 or 0) to one row per patient/lab timestamp. So if a patient has had three out of the ten
--top dx before their target lab test date, they should have one row with three 1's and not three rows w/ one 1 each
labels_dx_part2 AS
(
 
),

--identify all cases when cohort had one of the top 10 meds ordered before the target lab test was taken. 
--Merge so all meds for one patient/timestamp are in one row.
labels_dx_fam AS
(
  SELECT *
  FROM
  ( 
     SELECT anon_id,
      max(case when (family_hx.medical_hx like '%""" + hx1 + """%') then 1 else 0 end) as """ + hx_name1 + """_hist,
      max(case when (family_hx.medical_hx like '%""" + hx2 + """%') then 1 else 0 end) as """ + hx_name2 + """_hist,
      max(case when (family_hx.medical_hx like '%""" + hx3 + """%') then 1 else 0 end) as """ + hx_name3 + """_hist,
     FROM
      shc_core.family_hx
     GROUP BY
      anon_id
  ) as fam_hist
  RIGHT JOIN
  (
    SELECT *
    FROM labels_dx_part2
  ) USING(anon_id)
),

--get birthdate and gender
labels_dx_fam_demog AS
(
  SELECT *, 
  FROM
  (
    SELECT 
      anon_id, birth_date_jittered,
      (case when demographic.gender = "Male" then 1 else 0 end) as genderMale
    FROM
      shc_core.demographic
  )
  RIGHT JOIN
  (
    SELECT *
    FROM labels_dx_fam
  ) as hm
  USING(anon_id)
),

--identify all cases when cohort had one of the top 10 meds ordered before the target lab test was taken
labels_dx_fam_demog_meds AS
(  
  SELECT *
  FROM 
  (
    SELECT 
      anon_id as meds_anon_id, start_time_jittered,
      case when (order_med.med_description like '%""" + med1 + """%') then 1 else 0 end as """ + med_name1 + """_med,
      case when (order_med.med_description like '%""" + med2 + """%') then 1 else 0 end as """ + med_name2 + """_med,
      case when (order_med.med_description like '%""" + med3 + """%') then 1 else 0 end as """ + med_name3 + """_med,
      case when (order_med.med_description like '%""" + med4 + """%') then 1 else 0 end as """ + med_name4 + """_med,
      case when (order_med.med_description like '%""" + med5 + """%') then 1 else 0 end as """ + med_name5 + """_med,
      case when (order_med.med_description like '%""" + med6 + """%') then 1 else 0 end as """ + med_name6 + """_med,
      case when (order_med.med_description like '%""" + med7 + """%') then 1 else 0 end as """ + med_name7 + """_med,
      case when (order_med.med_description like '%""" + med8 + """%') then 1 else 0 end as """ + med_name8 + """_med,
      case when (order_med.med_description like '%""" + med9 + """%') then 1 else 0 end as """ + med_name9 + """_med,
      case when (order_med.med_description like '%""" + med10 + """%') then 1 else 0 end as """ + med_name10 + """_med,
    FROM
      shc_core.order_med
  ) AS meds
  RIGHT JOIN
  (
    SELECT 
      * except(birth_date_jittered), 
      DATETIME_DIFF(CAST(labels_taken_time as DATETIME), CAST(birth_date_jittered as DATETIME),YEAR) as age
      --use the date gained from the labels table with the birthdate to generate age
    FROM labels_dx_fam_demog
  ) AS labels
  ON meds.meds_anon_id = labels.anon_id
  AND CAST(meds.start_time_jittered as DATETIME) < DATETIME_SUB(CAST(labels.labels_taken_time as DATETIME), INTERVAL 1 DAY)
  AND CAST(meds.start_time_jittered as DATETIME) > DATETIME_SUB(CAST(labels.labels_taken_time as DATETIME), INTERVAL 6 MONTH)
),

--Merge so all meds for one patient/timestamp are in one row.
--note: couldn't do select * except b/c needed to modify by using aggregate functions
labels_dx_fam_demog_meds_v2 AS
(
  SELECT 
      anon_id as anon_id2, labels_taken_time, labels_result_time, label, age, """ + dx_name1 +"""_dx, """ + dx_name2 +"""_dx, """ + dx_name3 +"""_dx, 
      """ + dx_name4 +"""_dx, """ + dx_name5 +"""_dx, """ + dx_name6 +"""_dx, """ + dx_name7 +"""_dx, """ + dx_name8 +"""_dx, 
      """ + dx_name9 +"""_dx, """ + dx_name10 +"""_dx, genderMale, """ + hx_name1 +"""_hist, """ + hx_name2 +"""_hist, """ + hx_name3 +"""_hist,
      max(""" + med_name1 +"""_med) as """ + med_name1 +"""_med, max(""" + med_name2 +"""_med) as """ + med_name2 +"""_med, 
      max(""" + med_name3 +"""_med) as """ + med_name3 +"""_med, max(""" + med_name4 +"""_med) as """ + med_name4 +"""_med, 
      max(""" + med_name5 +"""_med) as """ + med_name5 +"""_med, max(""" + med_name6 +"""_med) as """ + med_name6 +"""_med, 
      max(""" + med_name7 +"""_med) as """ + med_name7 +"""_med, max(""" + med_name9 +"""_med) as """ + med_name8 +"""_med, 
      max(""" + med_name9 +"""_med) as """ + med_name9 +"""_med, max(""" + med_name10 +"""_med) as """ + med_name10 +"""_med
  FROM
    labels_dx_fam_demog_meds
  GROUP BY 
    anon_id2, labels_taken_time, labels_result_time, age, 
    """ + dx_name1 +"""_dx, """ + dx_name2 +"""_dx, """ + dx_name3 +"""_dx, """ + dx_name4 +"""_dx, """ + dx_name5 +"""_dx, """ + dx_name6 +"""_dx,
    """ + dx_name7 +"""_dx, """ + dx_name8 +"""_dx, """ + dx_name9 +"""_dx, """ + dx_name10 +"""_dx, genderMale, 
    """ + hx_name1 +"""_hist, """ + hx_name2 +"""_hist, """ + hx_name3 +"""_hist, label
),

--repeat of creating the labels table. Temporarily left bc of downstream usage of this table name. 
hs AS 
(   
  SELECT
    anon_id, taken_time, result_time,
    max(case when (lab_result.result_in_range_yn is null and lab_result.result_flag is not null) then 1 else 0 end) as label

  FROM
    shc_core.lab_result
  WHERE
    proc_code = '""" + proc_code + """' and not (reference_low is null and reference_high is null)
  GROUP BY
    anon_id, taken_time, result_time
),

--logic is a bit different from how hx, dx, and meds were found since we don't simply want to indicate 1/0 for
--having taken these feature lab tests or not. Instead start by reducing the shc_core labs table to only include
--the top ten labs 
labs AS
(
  SELECT 
    anon_id as labs_anon_id, lab_name as labs_lab_name, ord_value as lab_ord_value, 
    ord_num_value, reference_low, reference_high, taken_time as labs_taken_time
  FROM
    shc_core.lab_result
  WHERE 
    lab_result.ord_num_value != 9999999 and
    (
      lab_result.lab_name like '%""" + lab1_key + """%' or lab_result.lab_name like '%""" + lab2_key + """%' 
      or lab_result.lab_name like '%""" + lab3_key + """%' or lab_result.lab_name like '%""" + lab4_key + """%' 
      or lab_result.lab_name like '%""" + lab5_key + """%' or lab_result.lab_name like '%""" + lab6_key + """%' 
      or lab_result.lab_name like '%""" + lab7_key + """%' or lab_result.lab_name like '%""" + lab8_key + """%' 
      or lab_result.lab_name like '%""" + lab9_key + """%' or lab_result.lab_name like '%""" + lab10_key + """%' 
    )
  ORDER BY
    anon_id
),


--record the order value and time taken for each lab. Also only include lab tests taken within certain time period
--before the date the target lab test was taken. Note that there is some repetitive code here that will be
--streamlined in future iterations.
hs_labs AS
(
  SELECT *
  FROM 
  (
    SELECT 
      labs_anon_id, labs_taken_time, ord_num_value,
      case when labs.labs_lab_name like "%""" + lab1_key + """%" then labs.ord_num_value else null end as lab1_max,
      case when labs.labs_lab_name like "%""" + lab1_key + """%" then labs.ord_num_value else null end as lab1_min,
      case when labs.labs_lab_name like "%""" + lab1_key + """%" then labs.ord_num_value else null end as lab1_avg,
      case when labs.labs_lab_name like "%""" + lab1_key + """%" then labs.ord_num_value else 0 end as lab1_sum,
      case when labs.labs_lab_name like "%""" + lab1_key + """%" then labs.labs_taken_time else null end as lab1_recent_time,
      case when labs.labs_lab_name like "%""" + lab1_key + """%" then labs.labs_taken_time else null end as lab1_oldest_time,
      
      case when labs.labs_lab_name like "%""" + lab2_key + """%" then labs.ord_num_value else null end as lab2_max,
      case when labs.labs_lab_name like "%""" + lab2_key + """%" then labs.ord_num_value else null end as lab2_min,
      case when labs.labs_lab_name like "%""" + lab2_key + """%" then labs.ord_num_value else null end as lab2_avg,
      case when labs.labs_lab_name like "%""" + lab2_key + """%" then labs.ord_num_value else 0 end as lab2_sum,
      case when labs.labs_lab_name like "%""" + lab2_key + """%" then labs.labs_taken_time else null end as lab2_recent_time,
      case when labs.labs_lab_name like "%""" + lab2_key + """%" then labs.labs_taken_time else null end as lab2_oldest_time,
      
      case when labs.labs_lab_name like "%""" + lab3_key + """%" then labs.ord_num_value else null end as lab3_max,
      case when labs.labs_lab_name like "%""" + lab3_key + """%" then labs.ord_num_value else null end as lab3_min,
      case when labs.labs_lab_name like "%""" + lab3_key + """%" then labs.ord_num_value else null end as lab3_avg,
      case when labs.labs_lab_name like "%""" + lab3_key + """%" then labs.ord_num_value else 0 end as lab3_sum,
      case when labs.labs_lab_name like "%""" + lab3_key + """%" then labs.labs_taken_time else null end as lab3_recent_time,
      case when labs.labs_lab_name like "%""" + lab3_key + """%" then labs.labs_taken_time else null end as lab3_oldest_time,
      
      case when labs.labs_lab_name like "%""" + lab4_key + """%" then labs.ord_num_value else null end as lab4_max,
      case when labs.labs_lab_name like "%""" + lab4_key + """%" then labs.ord_num_value else null end as lab4_min,
      case when labs.labs_lab_name like "%""" + lab4_key + """%" then labs.ord_num_value else null end as lab4_avg,
      case when labs.labs_lab_name like "%""" + lab4_key + """%" then labs.ord_num_value else 0 end as lab4_sum,
      case when labs.labs_lab_name like "%""" + lab4_key + """%" then labs.labs_taken_time else null end as lab4_recent_time,
      case when labs.labs_lab_name like "%""" + lab4_key + """%" then labs.labs_taken_time else null end as lab4_oldest_time,
      
      case when labs.labs_lab_name like "%""" + lab5_key + """%" then labs.ord_num_value else null end as lab5_max,
      case when labs.labs_lab_name like "%""" + lab5_key + """%" then labs.ord_num_value else null end as lab5_min,
      case when labs.labs_lab_name like "%""" + lab5_key + """%" then labs.ord_num_value else null end as lab5_avg,
      case when labs.labs_lab_name like "%""" + lab5_key + """%" then labs.ord_num_value else 0 end as lab5_sum,
      case when labs.labs_lab_name like "%""" + lab5_key + """%" then labs.labs_taken_time else null end as lab5_recent_time,
      case when labs.labs_lab_name like "%""" + lab5_key + """%" then labs.labs_taken_time else null end as lab5_oldest_time,
      
      case when labs.labs_lab_name like "%""" + lab6_key + """%" then labs.ord_num_value else null end as lab6_max,
      case when labs.labs_lab_name like "%""" + lab6_key + """%" then labs.ord_num_value else null end as lab6_min,
      case when labs.labs_lab_name like "%""" + lab6_key + """%" then labs.ord_num_value else null end as lab6_avg,
      case when labs.labs_lab_name like "%""" + lab6_key + """%" then labs.ord_num_value else 0 end as lab6_sum,
      case when labs.labs_lab_name like "%""" + lab6_key + """%" then labs.labs_taken_time else null end as lab6_recent_time,
      case when labs.labs_lab_name like "%""" + lab6_key + """%" then labs.labs_taken_time else null end as lab6_oldest_time,
      
      case when labs.labs_lab_name like "%""" + lab7_key + """%" then labs.ord_num_value else null end as lab7_max,
      case when labs.labs_lab_name like "%""" + lab7_key + """%" then labs.ord_num_value else null end as lab7_min,
      case when labs.labs_lab_name like "%""" + lab7_key + """%" then labs.ord_num_value else null end as lab7_avg,
      case when labs.labs_lab_name like "%""" + lab7_key + """%" then labs.ord_num_value else 0 end as lab7_sum,
      case when labs.labs_lab_name like "%""" + lab7_key + """%" then labs.labs_taken_time else null end as lab7_recent_time,
      case when labs.labs_lab_name like "%""" + lab7_key + """%" then labs.labs_taken_time else null end as lab7_oldest_time,
      
      case when labs.labs_lab_name like "%""" + lab8_key + """%" then labs.ord_num_value else null end as lab8_max,
      case when labs.labs_lab_name like "%""" + lab8_key + """%" then labs.ord_num_value else null end as lab8_min,
      case when labs.labs_lab_name like "%""" + lab8_key + """%" then labs.ord_num_value else null end as lab8_avg,
      case when labs.labs_lab_name like "%""" + lab8_key + """%" then labs.ord_num_value else 0 end as lab8_sum,
      case when labs.labs_lab_name like "%""" + lab8_key + """%" then labs.labs_taken_time else null end as lab8_recent_time,
      case when labs.labs_lab_name like "%""" + lab8_key + """%" then labs.labs_taken_time else null end as lab8_oldest_time,
      
      case when labs.labs_lab_name like "%""" + lab9_key + """%" then labs.ord_num_value else null end as lab9_max,
      case when labs.labs_lab_name like "%""" + lab9_key + """%" then labs.ord_num_value else null end as lab9_min,
      case when labs.labs_lab_name like "%""" + lab9_key + """%" then labs.ord_num_value else null end as lab9_avg,
      case when labs.labs_lab_name like "%""" + lab9_key + """%" then labs.ord_num_value else 0 end as lab9_sum,
      case when labs.labs_lab_name like "%""" + lab9_key + """%" then labs.labs_taken_time else null end as lab9_recent_time,
      case when labs.labs_lab_name like "%""" + lab9_key + """%" then labs.labs_taken_time else null end as lab9_oldest_time,
      
      case when labs.labs_lab_name like "%""" + lab10_key + """%" then labs.ord_num_value else null end as lab10_max,
      case when labs.labs_lab_name like "%""" + lab10_key + """%" then labs.ord_num_value else null end as lab10_min,
      case when labs.labs_lab_name like "%""" + lab10_key + """%" then labs.ord_num_value else null end as lab10_avg,
      case when labs.labs_lab_name like "%""" + lab10_key + """%" then labs.ord_num_value else 0 end as lab10_sum,
      case when labs.labs_lab_name like "%""" + lab10_key + """%" then labs.labs_taken_time else null end as lab10_recent_time,
      case when labs.labs_lab_name like "%""" + lab10_key + """%" then labs.labs_taken_time else null end as lab10_oldest_time
      
      
      
    FROM 
      labs
  )AS labs_table
  INNER JOIN 
  (
    SELECT *
    FROM hs
  ) AS hs_table
  ON labs_table.labs_anon_id = hs_table.anon_id
  AND CAST(labs_table.labs_taken_time as DATETIME) <  DATETIME_SUB( CAST(hs_table.taken_time as DATETIME), INTERVAL 1 DAY) 
  AND CAST(labs_table.labs_taken_time as DATETIME) > DATETIME_SUB( CAST(hs_table.taken_time as DATETIME), INTERVAL 12 MONTH) 
),

--Aggregate to get the max, min, avg, and sum of the lab order values and to get the max and min of the time taken
lab_features AS
(
  SELECT
    anon_id, taken_time as hs_taken_time, result_time as hs_result_time,
    
    max(lab1_max) as lab1_max, min(lab1_min) as lab1_min, 
    avg(lab1_avg) as lab1_avg, sum(lab1_sum) as lab1_sum, 
    max(lab1_recent_time) as lab1_recent_time, min(lab1_oldest_time) as lab1_oldest_time,
    
    max(lab2_max) as lab2_max, min(lab2_min) as lab2_min, 
    avg(lab2_avg) as lab2_avg, sum(lab2_sum) as lab2_sum, 
    max(lab2_recent_time) as lab2_recent_time, min(lab2_oldest_time) as lab2_oldest_time,
    
    max(lab3_max) as lab3_max, min(lab3_min) as lab3_min, 
    avg(lab3_avg) as lab3_avg, sum(lab3_sum) as lab3_sum, 
    max(lab3_recent_time) as lab3_recent_time, min(lab3_oldest_time) as lab3_oldest_time,
    
    max(lab4_max) as lab4_max, min(lab4_min) as lab4_min, 
    avg(lab4_avg) as lab4_avg, sum(lab4_sum) as lab4_sum, 
    max(lab4_recent_time) as lab4_recent_time, min(lab4_oldest_time) as lab4_oldest_time,
    
    max(lab5_max) as lab5_max, min(lab5_min) as lab5_min, 
    avg(lab5_avg) as lab5_avg, sum(lab5_sum) as lab5_sum, 
    max(lab5_recent_time) as lab5_recent_time, min(lab5_oldest_time) as lab5_oldest_time,
    
    max(lab6_max) as lab6_max, min(lab6_min) as lab6_min, 
    avg(lab6_avg) as lab6_avg, sum(lab6_sum) as lab6_sum, 
    max(lab6_recent_time) as lab6_recent_time, min(lab6_oldest_time) as lab6_oldest_time,
    
    max(lab7_max) as lab7_max, min(lab7_min) as lab7_min, 
    avg(lab7_avg) as lab7_avg, sum(lab7_sum) as lab7_sum, 
    max(lab7_recent_time) as lab7_recent_time, min(lab7_oldest_time) as lab7_oldest_time,
    
    max(lab8_max) as lab8_max, min(lab8_min) as lab8_min, 
    avg(lab8_avg) as lab8_avg, sum(lab8_sum) as lab8_sum, 
    max(lab8_recent_time) as lab8_recent_time, min(lab8_oldest_time) as lab8_oldest_time,
    
    max(lab9_max) as lab9_max, min(lab9_min) as lab9_min, 
    avg(lab9_avg) as lab9_avg, sum(lab9_sum) as lab9_sum, 
    max(lab9_recent_time) as lab9_recent_time, min(lab9_oldest_time) as lab9_oldest_time,
    
    max(lab10_max) as lab10_max, min(lab10_min) as lab10_min, 
    avg(lab10_avg) as lab10_avg, sum(lab10_sum) as lab10_sum, 
    max(lab10_recent_time) as lab10_recent_time, min(lab10_oldest_time) as lab10_oldest_time,
    
  FROM
    hs_labs
  GROUP BY anon_id, taken_time, result_time
),

--Get the lab order values at the first (oldest) and last (recent) time the test was taken.
--If the same lab was taken multiple times on the first or last day, average the results.
lab_features_v2 AS
(
  SELECT 
    anon_id, hs_taken_time, 
    
    lab1_max, lab1_min, lab1_avg, 
    lab1_sum, lab1_recent_time, lab1_oldest_time,
    avg(case when lab1_ord_num_value is not null and labs_taken_time = lab1_recent_time then ord_num_value else null end) as lab1_recent_time_val,
    avg(case when lab1_ord_num_value is not null and labs_taken_time = lab1_oldest_time then ord_num_value else null end) as lab1_oldest_time_val,
    
    lab2_max, lab2_min, lab2_avg, 
    lab2_sum, lab2_recent_time, lab2_oldest_time,
    avg(case when lab2_ord_num_value is not null and labs_taken_time = lab2_recent_time then ord_num_value else null end) as lab2_recent_time_val,
    avg(case when lab2_ord_num_value is not null and labs_taken_time = lab2_oldest_time then ord_num_value else null end) as lab2_oldest_time_val,
    
    lab3_max, lab3_min, lab3_avg, 
    lab3_sum, lab3_recent_time, lab3_oldest_time,
    avg(case when lab3_ord_num_value is not null and labs_taken_time = lab3_recent_time then ord_num_value else null end) as lab3_recent_time_val,
    avg(case when lab3_ord_num_value is not null and labs_taken_time = lab3_oldest_time then ord_num_value else null end) as lab3_oldest_time_val,
    
    lab4_max, lab4_min, lab4_avg, 
    lab4_sum, lab4_recent_time, lab4_oldest_time,
    avg(case when lab4_ord_num_value is not null and labs_taken_time = lab4_recent_time then ord_num_value else null end) as lab4_recent_time_val,
    avg(case when lab4_ord_num_value is not null and labs_taken_time = lab4_oldest_time then ord_num_value else null end) as lab4_oldest_time_val,
    
    lab5_max, lab5_min, lab5_avg, 
    lab5_sum, lab5_recent_time, lab5_oldest_time,
    avg(case when lab5_ord_num_value is not null and labs_taken_time = lab5_recent_time then ord_num_value else null end) as lab5_recent_time_val,
    avg(case when lab5_ord_num_value is not null and labs_taken_time = lab5_oldest_time then ord_num_value else null end) as lab5_oldest_time_val,
    
    lab6_max, lab6_min, lab6_avg, 
    lab6_sum, lab6_recent_time, lab6_oldest_time,
    avg(case when lab6_ord_num_value is not null and labs_taken_time = lab6_recent_time then ord_num_value else null end) as lab6_recent_time_val,
    avg(case when lab6_ord_num_value is not null and labs_taken_time = lab6_oldest_time then ord_num_value else null end) as lab6_oldest_time_val,
    
    lab7_max, lab7_min, lab7_avg, 
    lab7_sum, lab7_recent_time, lab7_oldest_time,
    avg(case when lab7_ord_num_value is not null and labs_taken_time = lab7_recent_time then ord_num_value else null end) as lab7_recent_time_val,
    avg(case when lab7_ord_num_value is not null and labs_taken_time = lab7_oldest_time then ord_num_value else null end) as lab7_oldest_time_val,
    
    lab8_max, lab8_min, lab8_avg, 
    lab8_sum, lab8_recent_time, lab8_oldest_time,
    avg(case when lab8_ord_num_value is not null and labs_taken_time = lab8_recent_time then ord_num_value else null end) as lab8_recent_time_val,
    avg(case when lab8_ord_num_value is not null and labs_taken_time = lab8_oldest_time then ord_num_value else null end) as lab8_oldest_time_val,
    
    lab9_max, lab9_min, lab9_avg, 
    lab9_sum, lab9_recent_time, lab9_oldest_time,
    avg(case when lab9_ord_num_value is not null and labs_taken_time = lab9_recent_time then ord_num_value else null end) as lab9_recent_time_val,
    avg(case when lab9_ord_num_value is not null and labs_taken_time = lab9_oldest_time then ord_num_value else null end) as lab9_oldest_time_val,
    
    lab10_max, lab10_min, lab10_avg, 
    lab10_sum, lab10_recent_time, lab10_oldest_time,
    avg(case when lab10_ord_num_value is not null and labs_taken_time = lab10_recent_time then ord_num_value else null end) as lab10_recent_time_val,
    avg(case when lab10_ord_num_value is not null and labs_taken_time = lab10_oldest_time then ord_num_value else null end) as lab10_oldest_time_val,
     
  FROM
  (
    SELECT *
    FROM lab_features
    LEFT JOIN
    (
      --drop all cases where the lab order value was null
      SELECT 
        labs_anon_id, taken_time, ord_num_value, labs_taken_time, 
        case when lab1_max is not null then ord_num_value else null end as lab1_ord_num_value,
        case when lab2_max is not null then ord_num_value else null end as lab2_ord_num_value,
        case when lab3_max is not null then ord_num_value else null end as lab3_ord_num_value,
        case when lab4_max is not null then ord_num_value else null end as lab4_ord_num_value,
        case when lab5_max is not null then ord_num_value else null end as lab5_ord_num_value,
        case when lab6_max is not null then ord_num_value else null end as lab6_ord_num_value,
        case when lab7_max is not null then ord_num_value else null end as lab7_ord_num_value,
        case when lab8_max is not null then ord_num_value else null end as lab8_ord_num_value,
        case when lab9_max is not null then ord_num_value else null end as lab9_ord_num_value,
        case when lab10_max is not null then ord_num_value else null end as lab10_ord_num_value
        
      FROM hs_labs
    ) ON labs_anon_id = anon_id and taken_time = hs_taken_time
  )
  GROUP BY
    anon_id, hs_taken_time, 
    
    lab1_recent_time, lab1_oldest_time, lab1_max, 
    lab1_min, lab1_avg, lab1_sum,
    
    lab2_recent_time, lab2_oldest_time, lab2_max, 
    lab2_min, lab2_avg, lab2_sum,
    
    lab3_recent_time, lab3_oldest_time, lab3_max, 
    lab3_min, lab3_avg, lab3_sum,
    
    lab4_recent_time, lab4_oldest_time, lab4_max, 
    lab4_min, lab4_avg, lab4_sum,

    lab5_recent_time, lab5_oldest_time, lab5_max, 
    lab5_min, lab5_avg, lab5_sum,
    
    lab6_recent_time, lab6_oldest_time, lab6_max, 
    lab6_min, lab6_avg, lab6_sum,
    
    lab7_recent_time, lab7_oldest_time, lab7_max, 
    lab7_min, lab7_avg, lab7_sum,
    
    lab8_recent_time, lab8_oldest_time, lab8_max, 
    lab8_min, lab8_avg, lab8_sum,

    lab9_recent_time, lab9_oldest_time, lab9_max, 
    lab9_min, lab9_avg, lab9_sum,
    
    lab10_recent_time, lab10_oldest_time, lab10_max, 
    lab10_min, lab10_avg, lab10_sum
),

--use the oldest value, recent value, oldest time, and recent time to calculate the slope with the time difference
--in terms of months
lab_features_v3 AS
(
  SELECT 
    * except(
        lab1_recent_time, lab1_oldest_time, lab2_recent_time, lab2_oldest_time, 
        lab3_recent_time, lab3_oldest_time, lab4_recent_time, lab4_oldest_time, 
        lab5_recent_time, lab5_oldest_time, lab6_recent_time, lab6_oldest_time, 
        lab7_recent_time, lab7_oldest_time, lab8_recent_time, lab8_oldest_time, 
        lab9_recent_time, lab9_oldest_time, lab10_recent_time, lab10_oldest_time
        ),
    
    case when (lab1_oldest_time_val = lab1_recent_time_val) or DATETIME_DIFF(lab1_recent_time, lab1_oldest_time, MONTH) = 0 then 0 
      else (lab1_recent_time_val - lab1_oldest_time_val)/DATETIME_DIFF(lab1_recent_time, lab1_oldest_time, MONTH) end as lab1_slope,
      
    case when (lab2_oldest_time_val = lab2_recent_time_val) or DATETIME_DIFF(lab2_recent_time, lab2_oldest_time, MONTH) = 0 then 0 
      else (lab2_recent_time_val - lab2_oldest_time_val)/DATETIME_DIFF(lab2_recent_time, lab2_oldest_time, MONTH) end as lab2_slope,
      
    case when (lab3_oldest_time_val = lab3_recent_time_val) or DATETIME_DIFF(lab3_recent_time, lab3_oldest_time, MONTH) = 0 then 0 
      else (lab3_recent_time_val - lab3_oldest_time_val)/DATETIME_DIFF(lab3_recent_time, lab3_oldest_time, MONTH) end as lab3_slope,
      
    case when (lab4_oldest_time_val = lab4_recent_time_val) or DATETIME_DIFF(lab4_recent_time, lab4_oldest_time, MONTH) = 0 then 0 
      else (lab4_recent_time_val - lab4_oldest_time_val)/DATETIME_DIFF(lab4_recent_time, lab4_oldest_time, MONTH) end as lab4_slope,
      
    case when (lab5_oldest_time_val = lab5_recent_time_val) or DATETIME_DIFF(lab5_recent_time, lab5_oldest_time, MONTH) = 0 then 0 
      else (lab5_recent_time_val - lab5_oldest_time_val)/DATETIME_DIFF(lab5_recent_time, lab5_oldest_time, MONTH) end as lab5_slope,
         
   case when (lab6_oldest_time_val = lab6_recent_time_val) or DATETIME_DIFF(lab6_recent_time, lab6_oldest_time, MONTH) = 0 then 0 
      else (lab6_recent_time_val - lab6_oldest_time_val)/DATETIME_DIFF(lab6_recent_time, lab6_oldest_time, MONTH) end as lab6_slope,
        
    case when (lab7_oldest_time_val = lab7_recent_time_val) or DATETIME_DIFF(lab7_recent_time, lab7_oldest_time, MONTH) = 0 then 0 
      else (lab7_recent_time_val - lab7_oldest_time_val)/DATETIME_DIFF(lab7_recent_time, lab7_oldest_time, MONTH) end as lab7_slope,
        
    case when (lab8_oldest_time_val = lab8_recent_time_val) or DATETIME_DIFF(lab8_recent_time, lab8_oldest_time, MONTH) = 0 then 0 
      else (lab8_recent_time_val - lab8_oldest_time_val)/DATETIME_DIFF(lab8_recent_time, lab8_oldest_time, MONTH) end as lab8_slope,
       
    case when (lab9_oldest_time_val = lab9_recent_time_val) or DATETIME_DIFF(lab9_recent_time, lab9_oldest_time, MONTH) = 0 then 0 
      else (lab9_recent_time_val - lab9_oldest_time_val)/DATETIME_DIFF(lab9_recent_time, lab9_oldest_time, MONTH) end as lab9_slope,
          
   case when (lab10_oldest_time_val = lab10_recent_time_val) or DATETIME_DIFF(lab10_recent_time, lab10_oldest_time, MONTH) = 0 then 0 
      else (lab10_recent_time_val - lab10_oldest_time_val)/DATETIME_DIFF(lab10_recent_time, lab10_oldest_time, MONTH) end as lab10_slope,
         
  FROM 
    lab_features_v2
),

--join the lab features table with the dx/hx/meds features table
combined_table AS
(
  SELECT *
  FROM
    (
      labels_dx_fam_demog_meds_v2 INNER JOIN lab_features_v3 
      ON anon_id2 = anon_id AND labels_taken_time = hs_taken_time
    )
)

--return the combined feature table
SELECT * except(anon_id, hs_taken_time)
FROM combined_table
order by anon_id


""";
    return query
    

In [5]:
def queryData(query, dxFeature=False, totalFeatures=False):
    client = bigquery.Client("som-nero-phi-jonc101");
    if dxFeature:
        client = bigquery.Client("mining-clinical-decisions");
    conn = dbapi.connect(client);
    cursor = conn.cursor();
    cursor.execute(query);
    results_table = cursor.fetchall();
    
    if totalFeatures:
        df_all = (
                client.query(query)
                .result()
                .to_dataframe()
        )
        return df_all

    results_df = pd.DataFrame(results_table)
    results = []
    for i in range(len(results_df)):
        results.append(results_df[0][i][0])
            
    return results

In [6]:
def getData(proc_code):
    
    #get top labs
    lab_query = """
    SELECT lab_name, count(*)
    FROM 
      (
        select anon_id as labs_anon_id
        from shc_core.lab_result
        where proc_code = '""" + proc_code + """'
      ) LEFT JOIN
      (
        select anon_id, lab_name
        from shc_core.lab_result
      ) on labs_anon_id = anon_id
    WHERE lab_name is not null
    GROUP BY lab_name
    ORDER BY count(*) DESC
    limit 10
    """
    
    
    #get top diagnoses after transforming diagnosis codes into AHRQ Category Descriptions
    dx_query = """
    SELECT Category_Description, count(*)
    FROM
    (
      SELECT *
      FROM 
        (
          select anon_id as labs_anon_id
          from shc_core.lab_result
          where proc_code = '""" + proc_code + """'
        ) LEFT JOIN
        (
          select anon_id, icd10
          from shc_core.diagnosis_code
        ) on labs_anon_id = anon_id
    ) as dx 
    INNER JOIN
    (
      select ICD10, Category_Description 
      from ahrq_ccsr.diagnosis_code
    ) as codes 
    ON dx.icd10 = codes.ICD10
    WHERE Category_Description is not null
    GROUP BY Category_Description
    ORDER BY count(*) DESC
    limit 10
    """
  
    
    #get top histories
    hx_query = """
    SELECT medical_hx, count(*)
    FROM 
      (
        select anon_id as labs_anon_id
        from shc_core.lab_result
        where proc_code = '""" + proc_code + """'
      ) LEFT JOIN
      (
        select anon_id, medical_hx
        from shc_core.family_hx
      ) on labs_anon_id = anon_id
    WHERE medical_hx is not null
    GROUP BY medical_hx
    ORDER BY count(*) DESC
    limit 3
    """
   
    
    #get top medications
    med_query = """
    SELECT med_description, count(*)
    FROM 
      (
        select anon_id as labs_anon_id
        from shc_core.lab_result
        where proc_code = '""" + proc_code + """'
      ) LEFT JOIN
      (
        select anon_id, med_description
        from shc_core.order_med
      ) on labs_anon_id = anon_id
    WHERE med_description is not null
    GROUP BY med_description
    ORDER BY count(*) DESC
    limit 10
    """
   
    return queryData(lab_query), queryData(dx_query, True, False), queryData(hx_query), queryData(med_query) 

In [7]:
def processData(np_all):
    np_all = np_all[np.argsort(np_all[:, 1])] #sort by lab taken time so most recent tests are at bottom
    X_all = np_all[:,4:] #selects all except the row num, anon_id, timestamp, and labels columns
    Y_all = np_all[:,3] #selects the labels column
    
    N_train = round(len(X_all) * 0.8)
    
    X_train = X_all[:N_train, :]
    I_train = np.random.permutation(np.shape(X_train)[0])
    X_train = X_train[I_train, :] #shuffles the training data
    
    X_test = X_all[N_train:, :]
    I_test = np.random.permutation(np.shape(X_test)[0])
    X_test = X_test[I_test, :] #shuffles the test data
    
    Y_train = Y_all[:N_train]
    Y_train = Y_train[I_train]
    
    Y_test = Y_all[N_train:]
    Y_test = Y_test[I_test]
       
    return X_train, X_test, Y_train, Y_test

In [8]:
def balanceDataHelper(smallerClass, largerClass, X, Y):
    largeN = len(largerClass)
    upsampledSmall = np.random.choice(smallerClass, size=largeN, replace=True)
    X = np.concatenate((X[largerClass], X[upsampledSmall]))
    Y = np.concatenate((Y[largerClass], Y[upsampledSmall]))
    return X, Y
    
def balanceData(X_train, X_test, Y_train, Y_test):
    """
    Upsample the minority class by randomly selecting samples from the minority class until it has the same
    number of samples as the majority class
    """
    train0 = np.where(Y_train == 0)[0]
    train1 = np.where(Y_train == 1)[0]
    
    smallerClass = train0
    largerClass = train1
    if len(train0) > len(train1):
        smallerClass = train1
        largerClass = train0
    X_train, Y_train = balanceDataHelper(smallerClass, largerClass, X_train, Y_train)
        
    test0 = np.where(Y_test == 0)[0]
    test1 = np.where(Y_test == 1)[0]
    
    smallerClass = test0
    largerClass = test1
    if len(test0) > len(test1):
        smallerClass = test1
        largerClass = test0
    X_test, Y_test = balanceDataHelper(smallerClass, largerClass, X_test, Y_test)

    return X_train.astype(float), X_test.astype(float), Y_train.astype(int), Y_test.astype(int)

In [9]:
def runModel(X_train, Y_train):
    model = XGBClassifier(n_estimators = 300, learning_rate = 0.03, max_depth=5, subsample = 0.5)
    auc_cv_scores = cross_val_score(estimator = model, 
                                    X = X_train, 
                                    y = Y_train, 
                                    scoring = "roc_auc"
                                   )
    
    auc_avg = auc_cv_scores.mean()
    
    y_pred = cross_val_predict(model, X_train, Y_train)
    
    conf_mat = confusion_matrix(Y_train, y_pred)
    true_neg = conf_mat[0][0]
    false_pos = conf_mat[0][1]
    false_neg = conf_mat[1][0]
    TNR = true_neg / (true_neg + false_neg)
    NPV = true_neg / (true_neg + false_pos)

    return auc_avg, TNR, NPV

In [ ]:
results_dict = {} #key: lab name value: labs, dx, hx, meds, auroc, TNR, NPV

###CODE FOR USER TO MODIFY###
label_labs = ["LABTBIL", "LABA1C"] 
###END###

for proc_code in label_labs:
    labFeatures, dxFeatures, hxFeatures, medFeatures = getData(proc_code) #run query to select top 10 per ft. category
    features = [labFeatures, dxFeatures, hxFeatures, medFeatures]
    query = getQuery(proc_code, features) #dyanmically generate the feature matrix query
    if query is None
        continue
    
    df_all = queryData(query, False, True) #create feature feature matrix
    np_all = np.array(df_all.replace([None], np.nan))
    X_train, X_test, Y_train, Y_test = processData(np_all) #divide and shuffle data
    X_train, X_test, Y_train, Y_test = balanceData(X_train, X_test, Y_train, Y_test)
    
    #leave the test data untouched for now
    auc_avg, TNR, NPV = runModel(X_train, Y_train)
    
    results_dict[proc_code] = features + [auc_avg, TNR, NPV]
    
results_dict